# Coleta, Tratamento e Unificação das bases de dados relativas as Usinas Sucroalcooleiras

## Histórico de Atualizações

<table>
    <thead>
        <tr>
            <th>Versão</th>
            <th>Data</th>
            <th>Descrição</th>
            <th>Autor</th>
            <th>Email</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>1.0</th>
            <td>21/03/2023</td>
            <td>
                Tratamento das bases de dados.
                <ol>
                    <li>Cadastro de Instituicoes - MAPA</li>
                    <li>Autorizacoes Usinas de Biocombustives - ANP'</li>
                    <li>Autorizacoes Revogadas Usinas de Biocombustives - ANP'</li>
                    <li>Tancagens Etanol - ANP'</li>
                    <li>Marketing Share [Tancagem Diesel] - </li>
                    <li>Receita Operacional e Número de Empregados da Usinas - EMIS</li>
                </ol>
            </td>
            <td>Marcel Quintela, Sergio Urzedo Jr</td>
            <td>marcel.quintela@avanade.com, sergiourzedojr@gmail.com</td>
        </tr>
    </tbody>
</table>

# Setup Inicial

Carregando linhas com configurações iniciais já escritas em notebook base

No bloco de comando "%run" não pode ter nenhum outra linha de comando ou comentário

In [1]:
%run nb00_Setup_Usinas{'blob_relative_path_raw': 'nuvem/Usinas/'}      

# Leitura dos Dados 

In [2]:
files = [
         #'ANP/Usinas_Autorizadas/autorizacoes_biocombustiveis.xlsx', # arquivo com proteção tipo OLE2 usar xlwings
         'MAPA/Cadastro_Instituicoes/relatorio_completo_14032023010158.xls',
         'ANP/Usinas_Autorizadas/autorizacoes_biocombustiveis.csv',
         'ANP/Usinas_Autorizadas/Autorizacoes_biocombustives_revogadas.xlsx',
         'ANP/Tancagem/marco2023.csv',
         '/MM_Proj_OD.xlsx',
         '/usinas_enriquecidas.xlsx'
        ]

In [3]:
# NÃO FUNCIONOU BUSCAR FORMAS EFICAZES

# workbook = xw.Book(dir_m+files[0])
# sheet1 = workbook.sheets['Produtores de Etanol'].used_range.value
# df = pd.DataFrame(sheet1)


## MAPA - Cadastro de Instituições

O cadastro de usinas do MAPA foi definido em reunião com a  como a principal base de usinas, são elas que representarão as atuais usinas beneficiadoras de cana-de-açucar no Brasil

**Fonte**: Ministério da Agricultura

**URL**: https://sistemasweb4.agricultura.gov.br/sapcana/downloadBaseCompletaInstituicao.action

**Obs**: Esta base está sendo baixada manualmente, devido a proteção em nível 'captcha' impedindo o download automático.

In [4]:
inst = pd.read_excel(abfss_path_raw + files[0])

inst['CNPJ'] = [re.sub('[^a-zA-Z0-9 \\\]','',s) for s in inst['CNPJ']]          # ajuste de CNPJ remover caracts especiais
inst = inst[['CNPJ','Razão social','Cidade','UF','CEP','Produção Safra']]       # selecionando colunas a serem utilizadas
inst.columns = ['CNPJ','R_SOCIAL','MUNICIPIO','UF','CEP','PRODUCAO_SAFRA']


## ANP

### Usinas Autorizadas

Base de dados que auxilia no enriquecimento de informação da base de usinas.

**Fonte**: Agência Nacional 

**URL**: https://www.gov.br/anp/pt-br/assuntos/producao-e-fornecimento-de-biocombustiveis/autorizacao-para-producao-de-biocombustiveis/arquivos-autorizacao-para-producao-de-biocombustiveis/dados-autorizaoes-e-processos-produtores-biocombustiveis.zip

**Obs**: Arquivo compactado contendo arquivos de usinas:Autorizadas, Revogadas e em processo de Autorização.

In [5]:
col_names = pd.read_csv(abfss_path_raw + files[1], nrows=0).columns             # coletando informações de cabeçalho
col_types = {col: str for col in col_names if col not in {}}                    # todas as colunas como str

usinas_a = pd.read_csv(abfss_path_raw + files[1] , sep=";", dtype=col_types,
                        storage_options = {'linked_service' : linked_service_raw})

usinas_a.dropna(axis='columns',how='all', inplace=True)                         # limpando colunas vazias
usinas_a.columns = ['R_SOCIAL', 'CNPJ', 'MUNICIPIO','UF',                       # renomeando colunas
                  'CAP_HIDRATADO', 'CAP_ANIDRO', 
                  'AUTORIZA_AO', 'LINK_AO',
                  'AUTORIZA_AEA', 'LINK_AEA']                               


usinas_a['CNPJ'] = [re.sub('[^a-zA-Z0-9 \\\]','',s) for s in usinas_a['CNPJ']]    # ajuste de CNPJ remover caracts especiais
usinas_a = usinas_a[['CNPJ','R_SOCIAL', 'MUNICIPIO', 'UF',                       # selecionando colunas a serem utilizadas
                     'CAP_HIDRATADO','CAP_ANIDRO']]                   

del(col_names, col_types)

### Usinas Revogadas

As usinas revogadas tambem são utilizadas como enriquecimento de informações das usinas.



In [6]:
usinas_r  = pd.read_excel(abfss_path_raw + files[2], sheet_name='Produtores de Etanol', 
                            skiprows=5, usecols='A:B,D:F,H:I',
                            storage_options = {'linked_service' : linked_service_raw})
usinas_r.columns = ['R_SOCIAL', 'CNPJ', 'MUNICIPIO', 'UF', 'SITUACAO', 'CAP_ANIDRO', 'CAP_HIDRATADO']
usinas_r = usinas_r[['CNPJ','R_SOCIAL', 'MUNICIPIO', 'UF','CAP_HIDRATADO','CAP_ANIDRO','SITUACAO']]

usinas_r['CNPJ'] = usinas_r['CNPJ'].astype(str).apply(lambda x: x.zfill(14))  #Transforma CPNJ em string

### Unificar Usinas ANP

In [7]:
usinas_anp = usinas_a.merge(usinas_r, how='outer', on='CNPJ')

In [8]:
# preenchendo informações vindas da base MM_proj
usinas_anp['R_SOCIAL_x'] = usinas_anp['R_SOCIAL_x'].fillna(usinas_anp['R_SOCIAL_y'])
usinas_anp['MUNICIPIO_x'] = usinas_anp['MUNICIPIO_x'].fillna(usinas_anp['MUNICIPIO_y'])
usinas_anp['UF_x'] = usinas_anp['UF_x'].fillna(usinas_anp['UF_y'])
usinas_anp['CAP_HIDRATADO_x'] = usinas_anp['CAP_HIDRATADO_x'].fillna(usinas_anp['CAP_HIDRATADO_y'])
usinas_anp['CAP_ANIDRO_x'] = usinas_anp['CAP_ANIDRO_x'].fillna(usinas_anp['CAP_ANIDRO_y'])

usinas_anp['SITUACAO'] = usinas_anp['SITUACAO'].fillna('ATIVA')
usinas_anp.rename(columns={'SITUACAO': 'SITUACAO_ANP'}, inplace=True)

usinas_anp.drop(usinas_anp.filter(regex='_y').columns, axis=1, inplace=True)                            # Eliminando colunas com _y
usinas_anp.rename(columns=lambda s: s[:-2] if '_x' in s else s, inplace=True)                           # Renomeando colunas com _x 

### Tancagem

In [9]:
col_names = pd.read_csv(abfss_path_raw + files[3], nrows=0).columns         # coletando informações de cabeçalho
col_types = {col: str for col in col_names if col not in {}}                # todas as colunas como str

tanque = pd.read_csv(abfss_path_raw + files[3], dtype=col_types,
                        storage_options = {'linked_service' : linked_service_raw})

# ajuste das colunas para seus tipos originais
tanque['Data'] = pd.to_datetime(tanque['Data']) 
tanque['TancagemM3'] = tanque['TancagemM3'].astype(int)
tanque.rename(columns={'Cnpj':'CNPJ'}, inplace=True)

del(col_names, col_types)                                                   # eliminando dados de outra leitura

#### Filtrando infos do Setor

In [10]:
# Dados de tanque a serem considerados tancagem de volume de etanol produzido
tanque_g = tanque.groupby(['CNPJ','NomeEmpresarial','Municipio','Segmento','GrupoDeProdutos'])['TancagemM3'].sum().to_frame()
tanque_g.reset_index(inplace=True)
 
filtro_1 = (tanque_g['Segmento'] == 'INSTALAÇÃO PRODUTORA DE ETANOL')
filtro_2 = (tanque_g['GrupoDeProdutos']=='DERIVADOS E BIOCOMBUSTÍVEIS')

tanque_g = tanque_g[filtro_1 & filtro_2]

del(filtro_1, filtro_2)

## 

### Market Share Projeção OD

In [11]:
mm_proj = pd.read_excel(abfss_path_raw + files[4], usecols='B:I,R,Z',
                        storage_options = {'linked_service' : linked_service_raw})

In [12]:
mm_proj['CNPJ'] = mm_proj['CNPJ'].astype(str).apply(lambda x: x.zfill(14))  #Transforma CPNJ em string
mm_proj = mm_proj[~mm_proj['CNPJ'].str.startswith('000')]                   # removendo possiveis CPFs

In [13]:
mm_proj.drop_duplicates('CNPJ', keep='first', inplace=True)

## EMIS

### Receita Operacional e Empregados

In [14]:
usinas_emis = pd.read_excel(abfss_path_raw + files[5], sheet_name='emis', usecols='A,F,L,V', dtype=object,
                        storage_options = {'linked_service' : linked_service_raw})

usinas_emis.columns = ['EMPRESAS','RECEITA_OP','N_EMPREG', 'CNPJ']
usinas_emis = usinas_emis[[ 'CNPJ','EMPRESAS','RECEITA_OP','N_EMPREG']]

In [15]:
usinas_emis

In [16]:
def so_num(x):
    return(str(re.sub('[^0-9,]', "", x)))

In [17]:
# uniformizando as classes 
# x = pd.unique(usinas_emis['N_EMPREG1'].values)
# a = [s for s in x.astype('str') if '-' in s]
# b = [s for s in x.astype('str') if 'de' in s]
# c = a + b
# c

#### Organizando Receita Operacional

In [18]:
usinas_emis['RECEITA_OP1'] = ['9999999999' if x=='#' else x for x in usinas_emis['RECEITA_OP'].fillna('#')]                 # Fazendo ajuste dos NaN

usinas_emis['RECEITA_OP1'] = usinas_emis['RECEITA_OP1'].where(\
                                    usinas_emis['RECEITA_OP1']!='500.00 - 1,000',1000000)                                   # Alterando o valor exclusivo de 1,000 para 1Milhão

usinas_emis['RECEITA_OP1'] = [int(so_num(s.split(' - ')[-1].replace('.', '')))*10 \
                                    if '-' in s else int(float(s)*100) \
                                        for s in usinas_emis['RECEITA_OP1'].astype('string')]                               # Ajustando os valores numéricos

usinas_emis['RECEITA_OP_C'] = pd.cut(usinas_emis['RECEITA_OP1'].astype(int),                                                # Criando Categorias
                                        bins=[-1, 50, 250, 500, 1000, 2500, 5000, 
                                              10000, 25000, 50000, 100000,
                                              250000, 500000, 1000000],
                                        labels=['0-50','51-250', '251-500','501-1.000', '1.001-2.500', '2.501-5.000',
                                                '5.001-10.000','10.001-25.000','25.001-50.000','50.001-100.000',
                                                '100.001-250.000', '250.001-500.000', '500.001-1.000.000'])  

usinas_emis['RECEITA_OP_CC'] = usinas_emis['RECEITA_OP_C'].cat.codes                                                         # Codificando as Categorias

usinas_emis.drop('RECEITA_OP1', axis=1, inplace=True)                                                                        # Eliminando var de auxílio



# Cod abaixo executa o mesmo que a linha 6 -  list comprehension executa com melhor performance
# s = []
# for i in usinas_emis['RECEITA_OP1'].astype('str'):
#     if '-' in i:
#         a = int(so_num(i.split(' - ')[-1].replace('.', '')))*10
#     else:
#         a = int(float(i)*100)
#     s.append(a)


#### Organizando Nº de Empregados

In [19]:
usinas_emis['N_EMPREG1'] = ['999999999' if x=='#' else x[:-7] for x in usinas_emis['N_EMPREG'].fillna('#')]             # Removendo o Ano e fazendo ajuste dos NaN
usinas_emis['N_EMPREG1'] = usinas_emis['N_EMPREG1'].astype('str').str.replace(',','', regex=False)                      # Retirando a ',' como separador de milhar

usinas_emis['N_EMPREG1'] = [so_num(s[-3:]) if '-' in s else s for s in usinas_emis['N_EMPREG1']]                        # Associando o limite superior de classe a classe que tenha '-'
usinas_emis['N_EMPREG1'] = [so_num(s[-5:]) if 'de' in s else s for s in usinas_emis['N_EMPREG1']]                       # Associando o limite superior de classe a classe que tenha 'de'

usinas_emis['N_EMPREG_C'] = pd.cut(usinas_emis['N_EMPREG1'].astype(int), 
                                        bins=[-1, 0, 5, 9, 49, 199, 100000],
                                        labels=['<1','1-5', '6-9','10-49', '50-199', '>200'])                           # Criando Categorias

usinas_emis['N_EMPREG_CC']   = usinas_emis['N_EMPREG_C'].cat.codes                                                      # Codificando as Categorias

usinas_emis.drop('N_EMPREG1', axis=1, inplace=True)                                                                      # Eliminando var de auxílio


In [20]:
usinas_emis = usinas_emis[['CNPJ','RECEITA_OP','RECEITA_OP_C','RECEITA_OP_CC','N_EMPREG','N_EMPREG_C','N_EMPREG_CC']]

# Unificação das Bases

## 1ª Unificação das Bases
Base de dados de referência: **CADASTRO DE INSTITUIÇÕES - MAPA**

In [21]:
usinas_e = (
                inst[['CNPJ','R_SOCIAL','MUNICIPIO','UF','PRODUCAO_SAFRA']].merge(
                    usinas_anp, how='outer', on='CNPJ').merge(
                        tanque_g, how='left', on='CNPJ')
            )

# preenchendo informações vindas da usinas_anp
usinas_e['R_SOCIAL'] = usinas_e['R_SOCIAL_x'].fillna(usinas_e['R_SOCIAL_y'])
usinas_e['MUNICIPIO'] = usinas_e['MUNICIPIO_x'].fillna(usinas_e['MUNICIPIO_y'])
usinas_e['UF'] = usinas_e['UF_x'].fillna(usinas_e['UF_y'])

usinas_e = usinas_e[['CNPJ',
                     'R_SOCIAL',
                     'MUNICIPIO',
                     'UF',
                     'PRODUCAO_SAFRA',
                     'SITUACAO_ANP',
                     'CAP_HIDRATADO',
                     'CAP_ANIDRO',
                     'TancagemM3']]

usinas_e.rename(columns={'PRODUCAO_SAFRA':'PRODUCAO', 'TancagemM3':'TANCAGEM_ETANOL'}, inplace=True)

## 2ª Unificação das Bases

In [22]:
# Juntando as bases de Usinas MAPA enriquecida com infos da ANP de Autorização e Tancagem de Etanol 
# a base de Clientes do segmento Usinas do MM_Projeção da  
usinas_e2 = (
                usinas_e.merge(
                    mm_proj[['CNPJ','RSOCIAL', 'MUNIC','UF', 'Tancagem','AECO']], how='left', on='CNPJ').merge(
                        usinas_emis, how='left', on='CNPJ')       
            )

In [23]:
# preenchendo informações vindas da base MM_proj
usinas_e2['R_SOCIAL'] = usinas_e2['R_SOCIAL'].fillna(usinas_e2['RSOCIAL'])
usinas_e2['MUNICIPIO'] = usinas_e2['MUNICIPIO'].fillna(usinas_e2['MUNIC'])
usinas_e2['UF'] = usinas_e2['UF_x'].fillna(usinas_e2['UF_y'])

#usinas_e2['CLIENTE'] = usinas_e2['COD_SAP'].notnull().astype(int) # Se COD_SAP não nulo Verdade transformando para int da 1

#Mantendo somente as columas de interesse
colunas = ['CNPJ', 
           'R_SOCIAL',
           'UF_x',
           'MUNICIPIO',
           'PRODUCAO',
           'SITUACAO_ANP',
           'CAP_HIDRATADO',
           'CAP_ANIDRO',
           'TANCAGEM_ETANOL',
           'RECEITA_OP_C',
           'RECEITA_OP_CC',
           'N_EMPREG_C',
           'N_EMPREG_CC',
           'Tancagem',
           'AECO']

usinas = usinas_e2[colunas]
usinas.rename(columns={'UF_x':'UF', 'Tancagem':'TANCAGEM_DIESEL'}, inplace=True)

del(usinas_e2, usinas_e, usinas_a, usinas_r, usinas_anp, inst, tanque, tanque_g)

In [24]:
# Ajuste de informação float gravada como string
usinas['CAP_HIDRATADO'] = [x if type(x)==float else x.replace(",",'.') for x in  usinas['CAP_HIDRATADO']]
usinas['CAP_HIDRATADO'] = usinas['CAP_HIDRATADO'].astype(float)

# Ajuste de para maiusculas
#usinas = usinas.apply(lambda x: x.astype(str).str.upper())

In [26]:
477*4

# Salvando 

In [27]:
usinas.to_parquet(abfss_path_enriched + 'gov_usinas.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})